In [1]:
import requests
import json
import time
import copy

with open("key.txt", 'r') as f:
    key = f.read()
#print(key)

In [2]:
#This will only get us recent 500 matches
def retrieve_recent_match_data(key, start_at=None):
    if start_at is None:
        r = requests.get("https://api.steampowered.com/IDOTA2Match_570/GetMatchHistory/V001/?key=" + key + "&min_players=10")
        if r.status_code != 200:
            raise Exception("Servers might be down")
        else:
            return json.loads(r.text)
    else:
        r = requests.get("https://api.steampowered.com/IDOTA2Match_570/GetMatchHistory/V001/?start_at_match_id=" 
                         + str(start_at) + "&key=" + key + "&min_players=10")
        if r.status_code != 200:
            raise Exception("Either match id was bad or servers might be down")
        else:
            return json.loads(r.text)

In [3]:
private_id = 4294967295 #players that we can not get info from
unique_id = set([])
least_recent = None
leave_loop = False
while leave_loop is False:
    matches = retrieve_recent_match_data(key, start_at = least_recent)
    if matches['result']['results_remaining'] == 0:
        leave_loop = True
    for m in matches['result']['matches']:
        least_recent = m['match_id']
        for p in m['players']:
            if 'account_id' in p and p['account_id'] != private_id:
                unique_id.add(p['account_id'])
    time.sleep(0.9)
    print(len(unique_id))

271
535
811
1088
1370
1387


In [5]:
def find_player_recent_match(key, player_id, start_at = None):
    if start_at is None:
        r = requests.get("https://api.steampowered.com/IDOTA2Match_570/GetMatchHistory/V001/?key=" + key + 
                         "&min_players=10&account_id="+str(player_id))
        if r.status_code != 200:
            raise Exception("Servers might be down")
        else:
            return json.loads(r.text)
    else:
        r = requests.get("https://api.steampowered.com/IDOTA2Match_570/GetMatchHistory/V001/?start_at_match_id=" 
                         + str(start_at) + "&key=" + key + "&min_players=10&account_id="+str(player_id))
        if r.status_code != 200:
            raise Exception("Either match id was bad or servers might be down")
        else:
            return json.loads(r.text)

In [6]:
temp_set = copy.deepcopy(unique_id)
for account_id in temp_set:
    leave_loop = False
    least_recent = None
    while leave_loop is False:
        matches = find_player_recent_match(key, account_id, start_at = least_recent)
        if matches['result']['results_remaining'] == 0:
            leave_loop = True
        for m in matches['result']['matches']:
            least_recent = m['match_id']
            for p in m['players']:
                if 'account_id' in p and p['account_id'] != private_id:
                    unique_id.add(p['account_id'])
        time.sleep(0.9)
        print(len(unique_id))
    if len(unique_id) > 50000:
        break

1776
2158
2555
2946
3302
3313
3485
3800
4047
4176
4524
4908
5247
5713
6208
6231
6402
6538
6715
6830
7011
7019
7243
7555
7932
7986
8160
8310
8495
8698
9037
9054
9400
9776
10186
10569
10904
10919
11060
11197
11325
11492
11632
11637
12022
12410
12772
13143
13532
13549
13686
13822
14015
14271
14490
14505
14867
15227
15628
15973
16335
16353
16644
16877
17209
17544
17886
18191
18439
18470
18783
19142
19528
19985
20436
20460
21043
21488
21871
22240
22638
22652
22655
22869
23112
23329
23547
23792
23804
24020
24171
24332
24541
24791
24802
24966
25123
25304
25488
25701
25708
26236
26840
27463
27992
28563
28588
29268
29893
30521
31129
31749
31766
32288
32863
33378
33882
34349
34370
34557
34724
34884
35027
35223
35230
35804
36371
36929
37494
38020
38041
38402
38730
39097
39540
40050
40076
40537
40935
41344
41789
42204
42219
42268
42591
42925
43260
43553
43869
43883
44128
44332
44569
44811
45040
45047
45611
46162
46702
47245
47805
47826
48101
48353
48612
48866
49187
49202
49699
50168
50664
51201
51

In [7]:
with open("player_id.txt", 'w') as f:
    for p in unique_id:
        f.write(str(p))
        f.write("\n")

In [25]:
#Because of our limitations with the opendota api, we will only be parsing 100 player's word clouds, as we
#are limited to only 50,000 calls

#Nested dictionary, key1 = player_id, key2 = my_word_counts (things players has said), key3 = all_word_counts (things
#players have heard)
user_word_cloud = {}

def get_word_cloud(player_id):
    r = requests.get("https://api.opendota.com/api/players/" + str(player_id) +"/wordcloud")
    if r.status_code == 500:
        raise Exception("servers are down")
    if r.status_code != 200:
        return r.status_code
    else:
        return json.loads(r.text)
    
all_ids = list(unique_id)
counter = 0
for i in range(100):
    words = get_word_cloud(all_ids[i])
    if type(words) is int:
        print(str(all_ids[i]) + " caused a " + str(words) + " response")
        counter += 1
        continue
    elif len(words["my_word_counts"]) == 0:
        continue
    user_word_cloud[all_ids[i]] = words['my_word_counts']
    time.sleep(1.5)

In [23]:
with open("complete_word_cloud.json", 'w') as f:
    json.dump(user_word_cloud, f)

In [24]:
print(len(user_word_cloud))
print(counter) ##check to see how many failures we got

170
119


In [29]:
#http://norvig.com/spell-correct.html
#Download words
#import nltk
#nltk.download()

from nltk.corpus import words
word_list = words.words()